In [1]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal,beta,dirichlet, norm
from math import log
%matplotlib inline
plt.style.use('ggplot')
np.random.seed(1234)
from scipy.special import gamma
import ternary


np.set_printoptions(formatter={'all':lambda x: '%.3f' % x})

In [2]:
# a1 = 50
# a2 = 50
# a3 = 50
# beta = gamma(a1)*gamma(a2)*gamma(a3)/gamma(a1+a2+a3)

In [3]:
sz = 10
x1 = norm.rvs(-2,1,size=sz)
x2 = norm.rvs(2,1,size=sz)
x = np.hstack((x1,x2))

In [4]:
c_id = np.zeros(x.shape)
clusters,cluster_counts = np.unique(c_id,return_counts=True)
n_cluster = len(clusters)

sigma0 = 1
mu0 = 0

# sigma_cluster = np.array([sigma0])
# mu_cluster = np.array([mu0])

sigma_y = 1
tau_y = 1/(sigma_y**2)
ite = 30
alpha = 10

In [5]:
# rand_x = p0.rvs(size=2*sz)

# rand_x_prob = dirichlet.pdf(rand_x,size=2*sz)

# pred_x = norm.rvs(rand_x,sigma0,size=2*sz) * rand_x_prob

In [6]:
for _ in range(ite):
    for i in range(sz*2):
        
        assign_id = int(c_id[i])
        cluster_counts[assign_id] -= 1
        if cluster_counts[assign_id] == 0:
            cluster_counts[assign_id] = cluster_counts[-1]
            c_id[np.where(c_id==n_cluster-1)[0]] = assign_id
            clusters_counts = np.delete(cluster_counts, n_cluster-1) 
            n_cluster -= 1
        c_id[i] = -1
        print(assign_id)
        print(cluster_counts)

        logp = np.zeros((n_cluster+1))
        for j in range(n_cluster):

            idx = np.where(c_id == j)[0] # element indices within clusters[j]
            y_average = np.mean(x[idx])
            prior_tau_j = 1/(sigma0**2)
            mu_j = (y_average*cluster_counts[j]* tau_y + mu0 * prior_tau_j )/ (cluster_counts[j]* tau_y + prior_tau_j )
            sigma_j = np.sqrt(1/(cluster_counts[j]* tau_y + prior_tau_j ) + sigma_y**2)
            
            logp[j] = np.log(cluster_counts[j]/(sz*2-1+alpha)) + norm.logpdf(x[i],mu_j,sigma_j)
        new_cluster_mu = mu0
        new_cluster_sigma = np.sqrt(sigma0**2+sigma_y**2)
        
        logp[-1] = np.log(alpha/(sz*2-1+alpha)) + norm.logpdf(x[i],new_cluster_mu,new_cluster_sigma)
        print(logp)
        logp_max = np.max(logp)
        logp = logp - logp_max
        p = np.exp(logp)
        p = p/np.sum(p)
        print(p)        
        new_c = np.random.choice(np.arange(n_cluster+1),p=p)
        if new_c == n_cluster:
            # clusters = np.append(clusters,new_c)
            n_cluster += 1
            cluster_counts = np.append(cluster_counts,1)
        else:
            cluster_counts[new_c] += 1
        c_id[i] = new_c







0
[19.000]
[-2.576 -2.914]
[0.584 0.416]
0
[19.000]
[-6.676 -4.876]
[0.142 0.858]
0
[18.000 1.000]
[-1.691 -4.841 -2.411]
[0.654 0.028 0.318]
0
[17.000 1.000 1.000]
[-4.775 -4.660 -5.861 -3.667]
[0.182 0.204 0.062 0.552]
0
[16.000 1.000 1.000 1.000]
[-6.452 -4.911 -6.469 -5.305 -4.181]
[0.051 0.240 0.050 0.162 0.497]
0
[15.000 1.000 1.000 2.000]
[-2.997 -4.567 -4.718 -3.857 -2.640]
[0.309 0.064 0.055 0.131 0.441]
0
[14.000 1.000 1.000 2.000 1.000]
[-3.304 -4.558 -4.734 -3.845 -4.603 -2.655]
[0.233 0.066 0.056 0.136 0.064 0.446]
0
[13.000 1.000 2.000 2.000 1.000]
[-7.796 -4.850 -5.340 -4.082 -5.931 -4.068]
[0.008 0.158 0.097 0.340 0.053 0.344]
0
[12.000 1.000 2.000 3.000 1.000]
[-6.510 -4.539 -4.488 -3.301 -5.169 -3.315]
[0.015 0.104 0.110 0.360 0.056 0.355]
0
[11.000 1.000 2.000 3.000 1.000 1.000]
[-17.901 -6.825 -8.797 -5.462 -9.018 -8.011 -6.830]
[0.000 0.155 0.022 0.605 0.017 0.047 0.154]
0
[10.000 1.000 2.000 4.000 1.000 1.000]
[-3.271 -11.996 -8.924 -15.745 -9.068 -10.208 -4.811]


In [ ]:
print(cluster_counts)

In [ ]:
# start = 0
# scale = 10
# for i in range(start, scale + (1 - start)):
#     for j in range(start, scale + (1 - start) - i):
#         k = scale - i - j
#         print(i, j, k)

In [121]:
# # showcasing Dirichlet distribution
# def f(x):
#     return x[0]**(a1-1) * x[1]**(a2-1) * x[2]**(a3-1)/beta

# scale = 60
# fig, tax = ternary.figure(scale=scale)
# tax.heatmapf(f,scale=scale)

In [ ]:
# def shannon_entropy(p):
#     """Computes the Shannon Entropy at a distribution in the simplex."""
#     s = 0.
#     for i in range(len(p)):
#         try:
#             s += p[i] * log(p[i])
#         except ValueError:
#             continue
#     return -1.*s
# scale = 60

# figure, tax = ternary.figure(scale=scale)
# tax.heatmapf(shannon_entropy, boundary=True, style="triangular")
# tax.boundary(linewidth=2.0)
# tax.set_title("Shannon Entropy Heatmap")

# tax.show()